In [32]:
import pandas as pd
import numpy as np

path = 'C:/Users/elris/Dropbox/FBA/2020MiraeAsset/woogie/answer_sheet.csv'
answer_sheet = pd.read_csv(path)

path = 'C:/Users/elris/Dropbox/FBA/2020MiraeAsset/woogie/stocks.csv'
stocks = pd.read_csv(path, index_col = 0)

path = 'C:/Users/elris/Dropbox/FBA/2020MiraeAsset/woogie/trade_train.csv'
trade_train = pd.read_csv(path, index_col = 0)

In [33]:
group_list = list(set(trade_train['그룹번호']))
group_list = sorted(group_list)

In [34]:
is_top3 = set(trade_train['종목번호']).intersection(stocks[stocks['20년7월TOP3대상여부']=='Y']['종목번호'].unique())

In [35]:
for i in range(len(trade_train)) : 
    if trade_train['종목번호'][i] in is_top3 : 
        pass
    else : 
        trade_train = trade_train.drop(i, axis=0)

In [36]:
trade_train

,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값
0,201907,MAD01,288,A000660,Y,Y,7,17,19,234,74800,78500
2,201907,MAD01,288,A004020,Y,Y,3,3,409,528,41411,40000
3,201907,MAD01,288,A005380,Y,Y,4,3,323,451,140602,140628
4,201907,MAD01,288,A005930,Y,Y,13,19,34,101,45350,46850
5,201907,MAD01,288,A005935,Y,Y,5,6,49,204,37750,38878
...,...,...,...,...,...,...,...,...,...,...,...,...
30191,202006,MAD48,163,A096530,Y,Y,3,4,7,6,111100,118800
30192,202006,MAD48,163,A096770,Y,Y,4,5,11,19,125000,128000
30193,202006,MAD48,163,A105560,Y,Y,15,11,690,787,34999,35300
30196,202006,MAD48,163,A272210,Y,Y,3,3,90,42,9900,8870


In [38]:
trade_train[trade_train['그룹번호'] == 'MAD01']

,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값
0,201907,MAD01,288,A000660,Y,Y,7,17,19,234,74800,78500
2,201907,MAD01,288,A004020,Y,Y,3,3,409,528,41411,40000
3,201907,MAD01,288,A005380,Y,Y,4,3,323,451,140602,140628
4,201907,MAD01,288,A005930,Y,Y,13,19,34,101,45350,46850
5,201907,MAD01,288,A005935,Y,Y,5,6,49,204,37750,38878
...,...,...,...,...,...,...,...,...,...,...,...,...
25316,202006,MAD01,288,A090430,Y,Y,5,3,31,229,175000,157000
25317,202006,MAD01,288,A091990,Y,Y,3,4,132,162,94400,99009
25318,202006,MAD01,288,A096770,Y,Y,6,8,636,538,125479,125500
25319,202006,MAD01,288,A105560,Y,Y,4,6,27,51,38236,36100


In [37]:
df=pd.DataFrame()
tmp = 0
for group in group_list :
    for day in set(trade_train['기준년월']) :
        condition1 = trade_train['그룹번호'] == group
        condition2 = trade_train['기준년월'] == day

        df_ = trade_train[condition1 & condition2]
        df_ = df_.sort_values(by=['매수고객수','평균매수수량'], ascending =False)
        df_['점수'] = df_[:5]['매수고객수'].rank()
        df_ = df_[:5]
        if tmp == 0 :
            df = df_
        else : 
            df = pd.merge(df,df_, how='outer')
        tmp += 1
score_df = df.sort_values(by=['그룹번호','기준년월'], ascending=True)

In [20]:
score_df.head()

,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값,점수
10,201907,MAD01,288,A005930,Y,Y,13,19,34,101,45350,46850,5.0
11,201907,MAD01,288,A000660,Y,Y,7,17,19,234,74800,78500,4.0
12,201907,MAD01,288,A005935,Y,Y,5,6,49,204,37750,38878,3.0
13,201907,MAD01,288,A005380,Y,Y,4,3,323,451,140602,140628,2.0
14,201907,MAD01,288,A004020,Y,Y,3,3,409,528,41411,40000,1.0


In [23]:
for group in group_list :
    group_df = score_df[score_df['그룹번호'] == group]
    check_df = pd.DataFrame(index=set(group_df['기준년월']), columns=set(group_df['종목번호']))
    for day in set(group_df['기준년월']) :
        for stock in set(group_df['종목번호']) : 
            condition1= group_df['기준년월'] == day
            condition2= group_df['종목번호'] == stock
            try : 
                check_df[stock][day] = group_df[condition1 & condition2]['점수'].values[0]
            except IndexError : 
                pass
    check_df = check_df.sort_index()
    check_df.to_csv('C:/Users/elris/Dropbox/FBA/2020MiraeAsset/woogie/group/'+group+'_score.csv', index=True)

In [24]:
# 그룹별로 루프
tmp = 0
for group in group_list : 
    path = 'C:/Users/elris/Dropbox/FBA/2020MiraeAsset/woogie/score/'+group+'_score.csv'
    load_df = pd.read_csv(path, index_col=0)
    load_df = load_df.fillna(0)
    load_df = load_df.sort_index(ascending = False)
    
    # 단순지수평활(simple exponential smoothing) 가중치
    a = 0.8
    weight = []
    for i in range(len(load_df)) : 
        if i == 0 : 
            alpha = a
            weight.append(alpha)
        else : 
            alpha = alpha*(1-a)
            weight.append(alpha)
            
    score_dict = {}
    
    # 그룹별 상위 3개 종목 추출
    for i in range(len(score_df.columns)) :
        item_num = 0
        sum_score = load_df[load_df.columns[i]]@np.array(weight)
        score_dict[load_df.columns[i]] = sum_score
        item_num = sorted(score_dict.items(), key=lambda x: x[1], reverse=True)
    answer_sheet['종목번호1'][tmp] = item_num[0][0]
    answer_sheet['종목번호2'][tmp] = item_num[1][0]
    answer_sheet['종목번호3'][tmp] = item_num[2][0]
    tmp += 1

In [25]:
answer_sheet

,그룹명,종목번호1,종목번호2,종목번호3
0,MAD01,A005930,A000660,A005935
1,MAD02,A005930,A105560,A055550
2,MAD03,A019170,A003000,A005690
3,MAD04,A005930,A000660,A003000
4,MAD05,A035720,A035420,A068270
5,MAD06,A005930,A035720,A035420
6,MAD07,A005930,A096770,A010950
7,MAD08,A019170,A003000,A007570
8,MAD09,A035720,A005930,A051910
9,MAD10,A015760,A005935,A030200


In [26]:
answer_sheet.to_csv('C:/Users/elris/Dropbox/FBA/2020MiraeAsset/woogie/sol_sheet.csv', index=False)

In [27]:
len(set(trade_train['종목번호']))

133

In [28]:
len(score_df)

12

In [30]:
group_df

,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값,점수
2275,201907,MAD48,163,A005930,Y,Y,4,4,8,41,46600,47500,4.0
2276,201907,MAD48,163,A006800,Y,Y,3,3,550,674,8200,8010,2.5
2277,201907,MAD48,163,A000660,Y,Y,3,3,351,448,74539,75100,2.5
2278,201907,MAD48,163,A035420,N,Y,0,4,0,110,-1,118500,1.0
2279,201908,MAD48,163,A005930,Y,Y,9,3,73,50,43800,44250,2.0
2280,201908,MAD48,163,A090430,Y,Y,4,7,57,35,128000,130000,1.0
2281,201909,MAD48,163,A035720,Y,Y,4,3,67,118,134000,136500,3.0
2282,201909,MAD48,163,A051910,Y,Y,4,3,21,80,328250,328500,3.0
2283,201909,MAD48,163,A005930,Y,Y,4,5,18,36,48416,47300,3.0
2284,201909,MAD48,163,A000660,Y,Y,3,3,115,110,82200,82200,1.0


In [31]:
score_df

,A035720,A005380,A005490,A090430,A005385,A105560,A006400,A028300,A055550,A084990,...,A005930,A00680K,A096530,A035420,A006800,A067630,A000660,A009150,A028260,A002210
202006,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
202005,4.5,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,...,4.5,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0
202004,0.0,3.5,1.5,0.0,0.0,0.0,0.0,0.0,1.5,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0
202003,3.5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,3.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0
202002,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0
202001,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
201912,0.0,0.0,0.0,0.0,1.5,0.0,0.0,3.5,0.0,0.0,...,5.0,1.5,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0
201911,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,1.5,3.5,0.0,0.0,0.0
201910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,1.5,0.0,4.0,0.0
201909,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5
